# Download Weather Data from CDS

This notebook retrieves hourly ERA5 reanalysis fields and climate-
projection indicators from the
[Copernicus Climate Data Store (CDS)](https://cds.climate.copernicus.eu/).

**Contents**

| Section | Description |
|---------|-------------|
| 1 | Imports & helpers |
| 2 | ERA5 variable reference |
| 3 | ERA5 reanalysis download (single levels, 2015–2024) |
| 4 | Climate & energy projection download (SIS dataset) |

## 1. Imports & Helpers

In [ ]:
import cdsapi

# Convenience lists — avoid spelling out every month / day / hour.
ALL_MONTHS = [f"{m:02d}" for m in range(1, 13)]
ALL_DAYS   = [f"{d:02d}" for d in range(1, 32)]
ALL_HOURS  = [f"{h:02d}:00" for h in range(24)]

## 2. ERA5 Variable Reference

| Variable Name | ERA5 Official Name | Code | Unit | Description |
|---|---|---|---|---|
| pressure | Surface pressure | sp | Pa | Surface pressure |
| temperature | 2m temperature | t2m | K | Air temperature 2 m above the surface |
| soil temperature | Soil temperature level 4 | stl4 | K | Soil temperature between 1–3 m depth |
| influx_toa | TOA incident solar radiation | tisr | W/m² | Top-of-atmosphere incident solar radiation |
| influx_direct | Total sky direct solar radiation at surface | fdir | W/m² | Direct solar radiation at surface |
| runoff | Runoff | ro | m | Volume of water runoff per unit area |
| roughness | Forecast surface roughness | fsr | m | Surface roughness length |
| height | Geopotential | z | m | Surface elevation above sea level |
| albedo | Forecast albedo | fal | – | Diffuse-reflection ratio of solar radiation (0–1) |
| influx_diffuse | ssrd − fdir | ssrd − fdir | W/m² | Diffuse solar radiation at surface |
| wnd100m | 100 m wind speed | √(u100² + v100²) | m/s | Wind speed at 100 m (derived from u/v components) |

## 3. ERA5 Reanalysis — Single Levels (2015–2024)

Each variable group is downloaded as a separate NetCDF archive for
every year in the list.  The bounding box covers Europe
(N 65°, W −12°, S 33°, E 45°).

In [ ]:
DATASET_ERA5 = "reanalysis-era5-single-levels"

YEAR_LIST = list(range(2015, 2025))

VARIABLE_GROUPS = [
    ["10m_u_component_of_wind", "10m_v_component_of_wind"],
    ["100m_u_component_of_wind", "100m_v_component_of_wind"],
    ["2m_dewpoint_temperature", "2m_temperature"],
    ["forecast_albedo", "forecast_surface_roughness"],
    ["soil_temperature_level_4", "runoff"],
    ["toa_incident_solar_radiation",
     "surface_solar_radiation_downward_clear_sky"],
    ["surface_solar_radiation_downwards",
     "clear_sky_direct_solar_radiation_at_surface"],
    ["surface_runoff"],
]

EU_AREA = [65, -12, 33, 45]  # [north, west, south, east]

In [ ]:
client = cdsapi.Client()

for year in YEAR_LIST:
    for variables in VARIABLE_GROUPS:
        request = {
            "product_type": ["reanalysis"],
            "variable": variables,
            "year": [str(year)],
            "month": ALL_MONTHS,
            "day": ALL_DAYS,
            "time": ALL_HOURS,
            "data_format": "netcdf",
            "download_format": "zip",
            "area": EU_AREA,
        }
        client.retrieve(DATASET_ERA5, request).download()

## 4. Climate & Energy Projections (SIS Dataset)

Downloads climate and energy indicators for Europe derived from EURO-
CORDEX climate projections (RCP 4.5, CCLM4-8-17 driven by MPI-ESM-LR).

In [ ]:
DATASET_SIS = "sis-energy-derived-projections"

SIS_VARIABLES = [
    "wind_speed_at_100m",
    "wind_speed_at_10m",
    "surface_downwelling_shortwave_radiation",
    "2m_air_temperature",
]

client = cdsapi.Client()

for variable in SIS_VARIABLES:
    request = {
        "variable": [variable],
        "spatial_aggregation": "original_grid",
        "temporal_aggregation": "3_hourly",
        "experiment": ["rcp_4_5"],
        "rcm": "cclm4_8_17",
        "gcm": ["mpi_esm_lr"],
        "ensemble_member": ["r1i1p1"],
    }
    client.retrieve(DATASET_SIS, request).download()